# Instructions for Using the STAC API Extensions: query with more than one operator, sort, fields

This notebook provides a step-by-step guide on how to use the STAC (SpatioTemporal Asset Catalog) API to search for Sentinel-2 Level 2A data within a specified time range and geographic area. It demonstrates how to leverage STAC extensions for filtering, selecting, and sorting data based on specific parameters.

In the presented example, we will choose the Austrian Central Alps as the study area, focusing on the period from December 2017 to July 2018, in order to illustrate the process of snowmelt in mountainous regions, assuming low cloud cover to ensure the visualizations are clear and readable.

In this case, we will show how to use the `query` extension to construct the appropriate parameters for selecting data based on both cloud cover and snow cover conditions.

## Libraries

First, ensure all required packages are installed and imported.

In [1]:
import pystac_client
import geogif
import stackstac
import os

* `pystac_client`, `stackstack`: A Python library for working with STAC catalogs. It enables interaction with STAC services, reading catalog information, and searching for products based on specified parameters.
* `geogif`: A Python library for creating animated GIFs from data cubes, enabling the visualization of time series data in a dynamic and engaging format.
* `os`: A standard Python package that provides functionality for interacting with the operating system, including accessing environment variables and managing file paths. It is often used for authentication and setting up access credentials for various APIs and services, such as STAC.

## S3 Credentials

In order to access the data from CloudFerro's catalog, authentication is required. To facilitate this, you will need to create an account in the Copernicus Data Space Ecosystem (CDSE). The registration process is quick and can be completed in under 60 seconds. You can find the registration link here: https://documentation.dataspace.copernicus.eu/APIs/S3.html.

Once you have created an account, the same URL will redirect you to the section where you can generate your `S3 Credentials`. To do this, simply click the `Add Credential` button. This will allow you to define an expiration date for your credentials.

After selecting the expiration date, you will have successfully created your own S3 credentials. Please make sure to save the secret key at this point, as it will not be displayed again.

Now that you have your S3 credentials, you can use them to connect to the STAC catalog. By utilizing the `os` package, you will set environment variables to authenticate yourself as a valid user for the S3 protocol.

The only variables you need to configure are:
* `AWS_ACCESS_KEY_ID`
* `AWS_SECRET_ACCESS_KEY`.

In [2]:
os.environ["GDAL_HTTP_TCP_KEEPALIVE"] = "YES"
os.environ["AWS_S3_ENDPOINT"] = "eodata.dataspace.copernicus.eu"
os.environ["AWS_ACCESS_KEY_ID"] = "test"
os.environ["AWS_SECRET_ACCESS_KEY"] = "test"
os.environ["AWS_HTTPS"] = "YES"
os.environ["AWS_VIRTUAL_HOSTING"] = "FALSE"
os.environ["GDAL_HTTP_UNSAFESSL"] = "YES"

## Connecting to the STAC Catalog

We connect to the Copernicus STAC catalog, available at the endpoint: https://stac.dataspace.copernicus.eu/v1.

In [3]:
URL = "https://stac.dataspace.copernicus.eu/v1"
cat = pystac_client.Client.open(URL)
cat.add_conforms_to("ITEM_SEARCH")

## Defining the Area of Interest (AOI)

We define the area as a polygon based on geographical coordinates in GeoJSON format.
* `type`: Geometry type, in this case, a polygon.
* `coordinates`: A list of coordinates defining the area.

In [4]:
geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [10.9, 46.8],
            [11.3, 46.8],
            [11.3, 47.0],
            [10.9, 47.0],
            [10.9, 46.8],
        ]
    ],
}

## Defining Search Parameters

In this section, we define and combine parameters into a single dictionary for passing to the search function:
* `max_items`: The maximum number of results to retrieve.
* `collections`: The STAC collection (e.g., Sentinel-2 Level 2A data).
* `datetime`: Time range in YYYY-MM-DD/YYYY-MM-DD format.
* `intersects`: The previously defined area of interest.
* `query`: Typically a key-value pair where the key represents a specific property (e.g., eo:cloud_cover), and the value is the condition or comparison being applied.
* `sortby`: Sorts results (e.g., by the eo:cloud_cover property in ascending order).
* `fields`: Specifies which fields to exclude (e.g., geometry).

In [5]:
params = {
    "max_items": 100,
    "collections": "sentinel-2-l2a",
    "datetime": "2017-12-01/2018-07-30",
    "intersects": geom,
    "query": {"eo:cloud_cover": {"lte": 10}, "eo:snow_cover": {"gte": 0, "lte": 100}},
    "sortby": "properties.eo:snow_cover",
    "fields": {"exclude": ["geometry"]},
}

## Searching the Catalog

We query the catalog and convert the results into a list of dictionaries.

In [6]:
items = list(cat.search(**params).items_as_dicts())

## Creating DataCube from STAC

After performing the search, we can create a STACK object from the obtained items. In the definition of the STACK object, the following parameters must be declared:

* `items`: The STAC items that will be included in the stack object.
* `resolution`: The desired output resolution.
* `bounds_latlon`: The spatial bounding box of the output, given in degrees of latitude and longitude.
* `chunksize`: The chunk size to be used for the Dask array.
* `epsg`: The EPSG code for the coordinate reference system to which the data will be reprojected.
* `gdal_env`: GDAL configuration options to be used when opening and reading the datasets.

In [7]:
stack = stackstac.stack(
    items=items,
    resolution=(20, 20),
    bounds_latlon=(10.550, 47.200, 10.850, 47.400),
    chunksize=98304,
    epsg=32634,
    gdal_env=stackstac.DEFAULT_GDAL_ENV.updated(
        {
            "GDAL_NUM_THREADS": -1,
            "GDAL_HTTP_UNSAFESSL": "YES",
            "GDAL_HTTP_TCP_KEEPALIVE": "YES",
            "AWS_VIRTUAL_HOSTING": "FALSE",
            "AWS_HTTPS": "YES",
        }
    ),
)

## Band selection

In this step, we are selecting the specific spectral bands from the stacked satellite data that correspond to the Red, Green, and Blue (RGB) channels. These bands are essential for creating a true-color composite image, which represents the Earth’s surface in a way similar to how it would appear to the human eye.

The bands we select to be used to create an RGB image for visualisation, are:
* `B04_20m`: Red band
* `B03_20m`: Green band
* `B02_20m`: Blue band

In [8]:
rgb = stack.sel(band=["B04_20m", "B03_20m", "B02_20m"])

## Displaying results as a GIF

In this step, we create a GIF to visualize the filtered results. We obtain a sequence of images ordered by date in ascending order, presenting how the snow cover in the selected region of Alps changed during the specified time period with cloud cover less than 10%.

In [9]:
gif = geogif.dgif(rgb, fps=1).compute()
gif

RuntimeError: Error reading Window(col_off=0, row_off=0, width=1282, height=1262) from 's3://eodata/Sentinel-2/MSI/L2A_N0500/2018/01/29/S2A_MSIL2A_20180129T101251_N0500_R022_T32TPT_20230904T215802.SAFE/GRANULE/L2A_T32TPT_A013604_20180129T101247/IMG_DATA/R20m/T32TPT_20180129T101251_B03_20m.jp2': RasterioIOError('Read failed. See previous exception for details.')

## Displaying Snow Cover for Results

We extract the `eo:snow_cover` values for each item and display them. The previously used `sortby` extension allows for automatic sorting of the results based on the selected parameter, in this case, snow cover.

In [10]:
snow_cover_values = [item["properties"]["eo:snow_cover"] for item in items]
snow_cover_values

[6.59,
 6.89,
 29.89,
 34.03,
 35.42,
 35.9,
 39.35,
 39.52,
 40.87,
 42.57,
 44.36,
 44.55,
 44.83,
 45.71,
 46.19,
 46.98,
 46.99,
 48.32,
 49.29,
 51.8,
 52.55,
 55.29,
 62.63,
 69.95]